In [ ]:
devtools::install_github("hungms/annotatepip", dependencies = F)

In [2]:
setwd("/Users/hungm/Documents/development/annotatepip/")
library(annotatepip)

In [118]:
counts <- read.table("data/GSE224465.tsv", sep = "\t", header = T, row.names = 1)
counts[1:5, 1:5]
dim(counts)

,C.CB_R1,C.CB_R2,C.CB_R3,C.CB_R4,C.CC_R1
,<int>,<int>,<int>,<int>,<int>
0610005C13Rik,9,6,12,5,8
0610006L08Rik,0,0,0,0,0
0610009B22Rik,149,156,151,136,138
0610009E02Rik,3,3,4,0,2
0610009L18Rik,25,30,22,14,33


[1] 55291    29

## Remove selected genes from expression matrix

In [119]:
xy.genes <- get_xy_genes(org = "mouse")
xy.genes %>% head(.)

counts <- counts[!rownames(counts) %in% xy.genes,]
dim(counts)

[1] "Gm29289" "Ddx3y"   "Gm10256" "Sry"     "Kdm5d"   "Gm21677"

[1] 54380    29

In [120]:
mt.genes <- get_mt_genes(org = "mouse")
mt.genes %>% head(.)

counts <- counts[!rownames(counts) %in% mt.genes,]
dim(counts)

[1] "mt-Nd4" "mt-Nd1" "mt-Nd5" "mt-Nd6" "mt-Nd2" "mt-Nd3"

[1] 54367    29

In [121]:
vdj.genes <- get_str_genes(org = "mouse", str = c("tcr", "bcr"))
vdj.genes %>% head(.)

counts <- counts[!rownames(counts) %in% vdj.genes,]
dim(counts)

[1] "Ighv3-4"  "Ighv12-3" "Ighv1-55" "Ighv1-15" "Trdv5"    "Ighv1-22"

[1] 54102    29

## Convert gene orthologues

Convert mouse gene symbols to human.

In [148]:
human_genes <- convert_mouse_to_human(rownames(counts), one.to.many = F)
human_genes %>% head(.)

[1] "MT-ND4" "MT-ND1" "MT-ND5" "MT-ND6" "MT-ND2" "MT-ND3"

Convert human gene symbols to mouse.

In [150]:
mouse_genes <- convert_human_to_mouse(human_genes, one.to.many = F)
mouse_genes %>% head(.)

ERROR: Error in convert_human_to_mouse(human_genes, one.to.many = F): object 'moue.genes' not found


## Interoperability with GCT, GMT files

Save expression matrix as GCT file.

In [ ]:
write_gct(counts, save_name = "GSE224465.gct", save_dir = "tests/")

Load expression matrix from GCT file.

In [124]:
counts <- read_gct("tests/GSE224465.gct")
counts[1:5, 1:5]
dim(counts)

,C.CB_R1,C.CB_R2,C.CB_R3,C.CB_R4,C.CC_R1
,<int>,<int>,<int>,<int>,<int>
0610005C13Rik,9,6,12,5,8
0610006L08Rik,0,0,0,0,0
0610009B22Rik,149,156,151,136,138
0610009E02Rik,3,3,4,0,2
0610009L18Rik,25,30,22,14,33


[1] 54102    29

Save gene list as GMT file.

In [128]:
gene.list <- list(XY = xy.genes, MT = mt.genes, VDJ = vdj.genes)
gene.df <- convert_list_to_df(gene.list)
gene.df %>% head(.)

,XY,MT,VDJ
,<chr>,<chr>,<chr>
1,Gm29289,mt-Nd4,Ighv3-4
2,Ddx3y,mt-Nd1,Ighv12-3
3,Gm10256,mt-Nd5,Ighv1-55
4,Sry,mt-Nd6,Ighv1-15
5,Kdm5d,mt-Nd2,Trdv5
6,Gm21677,mt-Nd3,Ighv1-22


In [135]:
write_gmt(gene.df, save_name = "string_genes.gmt", save_dir = "tests/")

Load gene list from GMT file.

In [136]:
gmt <- read_gmt("tests/string_genes.gmt")
gmt %>% head(.)

,XY,MT,VDJ
,<chr>,<chr>,<chr>
V3,Gm29289,mt-Nd4,Ighv3-4
V4,Ddx3y,mt-Nd1,Ighv12-3
V5,Gm10256,mt-Nd5,Ighv1-55
V6,Sry,mt-Nd6,Ighv1-15
V7,Kdm5d,mt-Nd2,Trdv5
V8,Gm21677,mt-Nd3,Ighv1-22
